In [130]:
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import  silhouette_score
 


# from sklearn.metrics import f1_score, confusion_matrix
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier


# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Ridge ,Lasso
# from sklearn.model_selection import cross_val_score, GridSearchCV
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import RandomForestRegressor


# from xgboost import XGBRegressor


# from sklearn.metrics import accuracy_score
# import sklearn.metrics as metrics
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import classification_report



import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings


import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [131]:
df=pd.read_csv('/content/uber-raw-data-aug14.csv')

In [132]:
df.shape

(829275, 4)

In [133]:
df.head(6)

,Date/Time,Lat,Lon,Base
0,8/1/2014 0:03:00,40.7366,-73.9906,B02512
1,8/1/2014 0:09:00,40.7260,-73.9918,B02512
2,8/1/2014 0:12:00,40.7209,-74.0507,B02512
3,8/1/2014 0:12:00,40.7387,-73.9856,B02512
4,8/1/2014 0:12:00,40.7323,-74.0077,B02512
5,8/1/2014 0:13:00,40.7349,-74.0033,B02512


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829275 entries, 0 to 829274
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date/Time  829275 non-null  object 
 1   Lat        829275 non-null  float64
 2   Lon        829275 non-null  float64
 3   Base       829275 non-null  object 
dtypes: float64(2), object(2)
memory usage: 25.3+ MB


In [135]:
df['Base'].value_counts()

B02617    355803
B02598    220129
B02682    173280
B02764     48591
B02512     31472
Name: Base, dtype: int64

In [136]:
df.isnull().any()

Date/Time    False
Lat          False
Lon          False
Base         False
dtype: bool

In [137]:
data_sample =df.sample(20000)
data_sample.head()

,Date/Time,Lat,Lon,Base
455030,8/17/2014 19:55:00,40.7453,-73.9771,B02617
268860,8/2/2014 10:59:00,40.7240,-73.9499,B02617
204117,8/24/2014 10:46:00,40.7581,-73.9857,B02598
318183,8/6/2014 15:12:00,40.7154,-74.0139,B02617
223301,8/27/2014 16:51:00,40.8409,-73.9400,B02598


In [138]:
data_sample.loc[:, "Date/Time"] = pd.to_datetime(data_sample["Date/Time"])
data_sample.loc[:, "year"] = data_sample["Date/Time"].dt.year
data_sample.loc[:, "month"] = data_sample["Date/Time"].dt.month
data_sample.loc[:, "day"] = data_sample["Date/Time"].dt.day
data_sample.loc[:, "hour"] = data_sample["Date/Time"].dt.hour

weekdays_dict = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}
dataset=data_sample
dataset.loc[:, "weekday"] = dataset["Date/Time"].dt.weekday.map(weekdays_dict)
dataset = dataset.drop("Date/Time", axis = 1)

dataset.head()

,Lat,Lon,Base,year,month,day,hour,weekday
455030,40.7453,-73.9771,B02617,2014,8,17,19,Sunday
268860,40.7240,-73.9499,B02617,2014,8,2,10,Saturday
204117,40.7581,-73.9857,B02598,2014,8,24,10,Sunday
318183,40.7154,-74.0139,B02617,2014,8,6,15,Wednesday
223301,40.8409,-73.9400,B02598,2014,8,27,16,Wednesday


In [139]:
#la densité des B dans la map 
fig = px.scatter_mapbox(
        dataset, 
        lat="Lat", 
        lon="Lon",
        color="Base",
        mapbox_style="carto-positron",
        zoom=10
)

fig.show()
df_car = px.data.carshare()
df_car.head()

,centroid_lat,centroid_lon,car_hours,peak_hour
0,45.471549,-73.588684,1772.750000,2
1,45.543865,-73.562456,986.333333,23
2,45.487640,-73.642767,354.750000,20
3,45.522870,-73.595677,560.166667,23
4,45.453971,-73.738946,2836.666667,19


In [140]:
# Create pipeline for numeric features
numeric_features = ['Lon','Lat'] # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median',missing_values=np.nan)), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])
# Create pipeline for categorical features
categorical_features = ['Base','year','month','day','hour'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [141]:
print("Performing preprocessings on train set...")
print(dataset.head())
X_train = preprocessor.fit_transform(dataset)
print('...Done.')
print(X_train[0:5]) 
print()

Performing preprocessings on train set...
            Lat      Lon    Base  year  month  day  hour    weekday
455030  40.7453 -73.9771  B02617  2014      8   17    19     Sunday
268860  40.7240 -73.9499  B02617  2014      8    2    10   Saturday
204117  40.7581 -73.9857  B02598  2014      8   24    10     Sunday
318183  40.7154 -74.0139  B02617  2014      8    6    15  Wednesday
223301  40.8409 -73.9400  B02598  2014      8   27    16  Wednesday
...Done.
  (0, 0)	-0.11340196789879621
  (0, 1)	0.16397704049753756
  (0, 3)	1.0
  (0, 21)	1.0
  (0, 54)	1.0
  (1, 0)	0.3282014347304695
  (1, 1)	-0.3000118221303383
  (1, 3)	1.0
  (1, 6)	1.0
  (1, 45)	1.0
  (2, 0)	-0.25302657314192717
  (2, 1)	0.4428060283677319
  (2, 2)	1.0
  (2, 28)	1.0
  (2, 45)	1.0
  (3, 0)	-0.7108653949858162
  (3, 1)	-0.4873500483555187
  (3, 3)	1.0
  (3, 10)	1.0
  (3, 50)	1.0
  (4, 0)	0.48893208495221857
  (4, 1)	2.2464810436532257
  (4, 2)	1.0
  (4, 31)	1.0
  (4, 51)	1.0



In [142]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_train)
    wcss.append(kmeans.inertia_)
    
print(wcss)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



[79104.41600112827, 71558.66778208935, 66767.18441887795, 63140.24675630884, 59996.499651515696, 57997.86246469233, 55745.18618389807, 54481.75434847704, 53037.33726724848]


In [143]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [144]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_train)
    s_score.append(silhouette_score(X_train, kmeans.predict(X_train)))

print(s_score)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

[0.3581174795715048, 0.09928130494469371, 0.10150007678325212, 0.1321053167363211, 0.12624465638737792, 0.1295340970979958, 0.13252349339134145, 0.13377159331893818, 0.13210776208256927]


In [145]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show()

In [146]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 8)
kmeans.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans()

In [147]:
dataset.loc[:,'Cluster_KMeans'] = kmeans.predict(X_train)
dataset.head()

,Lat,Lon,Base,year,month,day,hour,weekday,Cluster_KMeans
455030,40.7453,-73.9771,B02617,2014,8,17,19,Sunday,0
268860,40.7240,-73.9499,B02617,2014,8,2,10,Saturday,0
204117,40.7581,-73.9857,B02598,2014,8,24,10,Sunday,4
318183,40.7154,-74.0139,B02617,2014,8,6,15,Wednesday,0
223301,40.8409,-73.9400,B02598,2014,8,27,16,Wednesday,3


In [148]:
# Visualisation bi-dimensionnelle
fig = px.scatter(dataset, x = 'day', y = "hour", color = "Cluster_KMeans")
fig.show()

In [151]:
px.scatter_mapbox(
    dataset,
    lat="Lat",
    lon="Lon",
    color="Cluster_KMeans",
    mapbox_style="carto-positron",
)



In [152]:
dataset.head()

,Lat,Lon,Base,year,month,day,hour,weekday,Cluster_KMeans
455030,40.7453,-73.9771,B02617,2014,8,17,19,Sunday,0
268860,40.7240,-73.9499,B02617,2014,8,2,10,Saturday,0
204117,40.7581,-73.9857,B02598,2014,8,24,10,Sunday,4
318183,40.7154,-74.0139,B02617,2014,8,6,15,Wednesday,0
223301,40.8409,-73.9400,B02598,2014,8,27,16,Wednesday,3


# ***work on specific day wednesday***

In [153]:
dWednesday=dataset.loc[(dataset['weekday']=='Wednesday')]
dWednesday

,Lat,Lon,Base,year,month,day,hour,weekday,Cluster_KMeans
318183,40.7154,-74.0139,B02617,2014,8,6,15,Wednesday,0
223301,40.8409,-73.9400,B02598,2014,8,27,16,Wednesday,3
315590,40.7513,-74.0066,B02617,2014,8,6,10,Wednesday,0
408787,40.7586,-73.9742,B02617,2014,8,13,21,Wednesday,0
721293,40.7204,-74.0010,B02682,2014,8,20,18,Wednesday,5
...,...,...,...,...,...,...,...,...,...
175739,40.6929,-73.9909,B02598,2014,8,20,16,Wednesday,2
73857,40.7615,-73.9751,B02598,2014,8,6,15,Wednesday,4
21137,40.7643,-73.9733,B02512,2014,8,20,17,Wednesday,4
400470,40.8183,-73.9410,B02617,2014,8,13,9,Wednesday,3


In [156]:
px.scatter_mapbox(
    dWednesday,
    lat="Lat",
    lon="Lon",
    color="Cluster_KMeans",
    size='hour',
    mapbox_style="carto-positron",
    title=' vis for Wednesday'
)



# ***specific hour***

In [157]:
dWednesday=dataset.loc[(dataset['weekday']=='Wednesday')]

In [160]:
px.scatter_mapbox(
    dataset.loc[(dataset['hour']==12)], #no to outliers
    lat="Lat",
    lon="Lon",
    color="Cluster_KMeans",
    size='hour',
    mapbox_style="carto-positron",
    title=' vis at 12pm'
)



# ***DBSCAN***

In [161]:
# import DBSCAN from sklearn 
from sklearn.cluster import DBSCAN


# Instanciate DBSCAN 
db = DBSCAN(eps=0.1, min_samples=5, metric="euclidean", algorithm="brute")

# Fit on data 
## No need to normalize data, it already is! 
db.fit(X_train)


DBSCAN(algorithm='brute', eps=0.1)

In [162]:
np.unique(db.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17])

In [163]:
dataset["cluster"] = db.labels_
dataset.head()

,Lat,Lon,Base,year,month,day,hour,weekday,Cluster_KMeans,cluster
455030,40.7453,-73.9771,B02617,2014,8,17,19,Sunday,0,-1
268860,40.7240,-73.9499,B02617,2014,8,2,10,Saturday,0,-1
204117,40.7581,-73.9857,B02598,2014,8,24,10,Sunday,4,-1
318183,40.7154,-74.0139,B02617,2014,8,6,15,Wednesday,0,-1
223301,40.8409,-73.9400,B02598,2014,8,27,16,Wednesday,3,-1


In [173]:
type(X_train)

scipy.sparse._csr.csr_matrix

In [176]:
# Visualize with plotly 

## Import go to build our own figure
import plotly.graph_objects as go

fig = go.Figure()

# Loop through each label for our cluster
for i in np.unique(db.labels_):
    label = X_train.toarray()[db.labels_ == i]
    fig.add_trace(go.Scatter(x=label[:, 0], y=label[:, 1], mode="markers", name="Cluster {}".format(i)))

fig.show()

# ***it depends on  how strong ur pc to look for best params ***

In [182]:
px.scatter_mapbox(
    dataset.loc[dataset['cluster'] != -1, :],
    lat="Lat",
    lon="Lon",
    color="cluster",
    mapbox_style="carto-positron",
    zoom=10
)